<a href="https://colab.research.google.com/github/ilunga-k/AI-study/blob/main/Colab_%EC%8B%9C%EC%9E%91%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- 업로드 & 압축 해제 ---
import os, glob, zipfile, shutil
from google.colab import files

BASE = "/content"
HAND_ROOT = f"{BASE}/hand_dataset"  # 표준 위치

print("hand 데이터셋 zip 파일을 선택해줘.")
up = files.upload()  # 예: hand_dataset.zip
zip_path = os.path.join(BASE, list(up.keys())[0])

# 기존 폴더 있으면 제거
if os.path.exists(HAND_ROOT):
    shutil.rmtree(HAND_ROOT)

with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(BASE)

# --- 루트 자동 탐지: train/val(또는 vaild/valid) 안에 images/labels가 있는 폴더 찾기 ---
def looks_like_root(p):
    def ok(sp):
        return os.path.isdir(f"{p}/{sp}/images") and os.path.isdir(f"{p}/{sp}/labels")
    # train 은 필수, 검증은 val / valid / vaild 중 하나만 있어도 OK
    return ok("train") and any(ok(v) for v in ["val","valid","vaild"])

cands = [d for d in glob.glob(f"{BASE}/*") if os.path.isdir(d)]
src_root = next((d for d in sorted(cands, key=len) if looks_like_root(d)), None)
assert src_root, "데이터 루트를 찾지 못했어. zip 내부 구조(…/train/images, …/train/labels)를 확인해줘."

# 표준 경로로 이동
shutil.move(src_root, HAND_ROOT)

# --- 오탈자 폴더명 교정: valid, vaild -> val ---
for wrong in ["valid","vaild"]:
    if os.path.exists(f"{HAND_ROOT}/{wrong}") and not os.path.exists(f"{HAND_ROOT}/val"):
        shutil.move(f"{HAND_ROOT}/{wrong}", f"{HAND_ROOT}/val")

# 최종 구조 확인
assert os.path.isdir(f"{HAND_ROOT}/train/images") and os.path.isdir(f"{HAND_ROOT}/train/labels"), "train 폴더 구조 오류"
assert os.path.isdir(f"{HAND_ROOT}/val/images")   and os.path.isdir(f"{HAND_ROOT}/val/labels"),   "val 폴더 구조 오류"
print("✅ hand 데이터셋 구조 OK →", HAND_ROOT)


hand 데이터셋 zip 파일을 선택해줘.


Saving hand_dataset.zip to hand_dataset.zip
✅ hand 데이터셋 구조 OK → /content/hand_dataset


In [5]:
import os, zipfile, urllib.request

BASE = "/content"
COCO_DIR = f"{BASE}/coco2017"
os.makedirs(COCO_DIR, exist_ok=True)

files = {
    "train_images": ("http://images.cocodataset.org/zips/train2017.zip", f"{COCO_DIR}/train2017.zip"),
    "val_images":   ("http://images.cocodataset.org/zips/val2017.zip",   f"{COCO_DIR}/val2017.zip"),
    "ann":          ("http://images.cocodataset.org/annotations/annotations_trainval2017.zip", f"{COCO_DIR}/ann.zip"),
}
for name, (url, path) in files.items():
    if not os.path.exists(path):
        print(f"downloading {name} ...")
        urllib.request.urlretrieve(url, path)

def unzip(p, to):
    with zipfile.ZipFile(p, 'r') as zf: zf.extractall(to)

if not os.path.isdir(f"{COCO_DIR}/train2017"): unzip(files["train_images"][1], COCO_DIR)
if not os.path.isdir(f"{COCO_DIR}/val2017"):   unzip(files["val_images"][1],   COCO_DIR)
if not os.path.isdir(f"{COCO_DIR}/annotations"):unzip(files["ann"][1],          COCO_DIR)
print("COCO ready:", os.listdir(COCO_DIR))


downloading train_images ...
downloading val_images ...
downloading ann ...
COCO ready: ['annotations', 'val2017', 'val2017.zip', 'ann.zip', 'train2017', 'train2017.zip']


In [6]:
import json, shutil, math
from pycocotools.coco import COCO

# 생성될 가방 전용 데이터셋
ITEMS_ROOT = f"{BASE}/items_dataset"
for sp in ["train","val"]:
    os.makedirs(f"{ITEMS_ROOT}/{sp}/images", exist_ok=True)
    os.makedirs(f"{ITEMS_ROOT}/{sp}/labels", exist_ok=True)

# 전역 클래스 매핑(꼭 유지)
GLOBAL_IDX = {"backpack":1, "handbag":2, "suitcase":3}

# 변환 파라미터
MIN_AREA_RATIO = 0.005   # 프레임 대비 너무 작은 박스 제거(선택)
INCLUDE_NEGATIVES = False # True면 타겟없는 이미지에 빈 라벨 파일 생성(오탐 억제용)

def coco_to_yolo_split(coco_json_path, img_dir, out_split_dir):
    coco = COCO(coco_json_path)
    # COCO catId 매핑
    name_to_cat = {cat["name"]:cat["id"] for cat in coco.loadCats(coco.getCatIds())}
    target_cats = [name_to_cat[n] for n in ["backpack","handbag","suitcase"] if n in name_to_cat]

    # 이미지 메타
    imgs = {img["id"]:img for img in coco.loadImgs(coco.getImgIds())}

    # 이미지별 라벨 누적
    img_to_lines = {img_id: [] for img_id in imgs.keys()}
    ann_ids = coco.getAnnIds(catIds=target_cats, iscrowd=None)
    anns = coco.loadAnns(ann_ids)

    for a in anns:
        img_id = a["image_id"]
        img_meta = imgs[img_id]
        W, H = img_meta["width"], img_meta["height"]
        x, y, w, h = a["bbox"]  # COCO: 좌상단 x,y, w,h (픽셀)
        if w <= 1 or h <= 1:
            continue
        if (w*h)/(W*H) < MIN_AREA_RATIO:
            continue
        # 정규화된 YOLO(cx,cy,w,h)
        cx = (x + w/2) / W
        cy = (y + h/2) / H
        nw = w / W
        nh = h / H

        cat_name = coco.loadCats([a["category_id"]])[0]["name"]
        cls = GLOBAL_IDX.get(cat_name, None)
        if cls is None:
            continue
        img_to_lines[img_id].append(f"{cls} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")

    # 라벨/이미지 저장
    save_img_dir = f"{out_split_dir}/images"
    save_lbl_dir = f"{out_split_dir}/labels"

    pos_cnt = 0
    for img_id, meta in imgs.items():
        fname = meta["file_name"]
        src = os.path.join(img_dir, fname)
        dst = os.path.join(save_img_dir, fname)

        lines = img_to_lines[img_id]
        if len(lines)==0 and not INCLUDE_NEGATIVES:
            continue  # 타겟 없는 이미지는 스킵(기본)
        # 이미지 복사
        if not os.path.exists(dst):
            shutil.copy2(src, dst)
        # 라벨 저장(비어있으면 빈 파일)
        lbl_path = os.path.join(save_lbl_dir, os.path.splitext(fname)[0] + ".txt")
        with open(lbl_path, "w") as f:
            f.write("\n".join(lines))
        if lines: pos_cnt += 1

    total_imgs = len([f for f in os.listdir(save_img_dir) if f.lower().endswith(('.jpg','.jpeg','.png'))])
    print(f"[{out_split_dir}] 라벨 있는 이미지: {pos_cnt}, 저장된 이미지 총계: {total_imgs}")

# 실행
coco_to_yolo_split(f"{COCO_DIR}/annotations/instances_train2017.json", f"{COCO_DIR}/train2017", f"{ITEMS_ROOT}/train")
coco_to_yolo_split(f"{COCO_DIR}/annotations/instances_val2017.json",   f"{COCO_DIR}/val2017",   f"{ITEMS_ROOT}/val")

# 단독 테스트용 items 전용 data.yaml도 만들어 둠
open(f"{ITEMS_ROOT}/data_items.yaml","w").write(f"""\
path: {ITEMS_ROOT}
train: train/images
val: val/images
nc: 4
names: [hand, backpack, handbag, suitcase]
""")
print(open(f"{ITEMS_ROOT}/data_items.yaml").read())


loading annotations into memory...
Done (t=32.74s)
creating index...
index created!
[/content/items_dataset/train] 라벨 있는 이미지: 8823, 저장된 이미지 총계: 8823
loading annotations into memory...
Done (t=0.91s)
creating index...
index created!
[/content/items_dataset/val] 라벨 있는 이미지: 364, 저장된 이미지 총계: 364
path: /content/items_dataset
train: train/images
val: val/images
nc: 4
names: [hand, backpack, handbag, suitcase]



In [4]:
import pathlib

def force_class_zero(lbl_dir):
    n_edit = 0
    for p in pathlib.Path(lbl_dir).glob("*.txt"):
        lines_out = []
        changed = False
        for ln in p.read_text().splitlines():
            ln = ln.strip()
            if not ln:
                continue
            parts = ln.split()
            if not parts:
                continue
            if parts[0] != "0":
                parts[0] = "0"
                changed = True
            lines_out.append(" ".join(parts))
        p.write_text("\n".join(lines_out))
        if changed: n_edit += 1
    return n_edit

edited_tr = force_class_zero(f"{HAND_ROOT}/train/labels")
edited_va = force_class_zero(f"{HAND_ROOT}/val/labels")
print(f"hand 라벨 강제 통일 완료: train 수정 {edited_tr}개, val 수정 {edited_va}개")


hand 라벨 강제 통일 완료: train 수정 0개, val 수정 0개


In [7]:
ITEMS_ROOT = f"{BASE}/items_dataset"
assert os.path.isdir(f"{ITEMS_ROOT}/train/images") and os.path.isdir(f"{ITEMS_ROOT}/train/labels"), \
    "items_dataset이 없어. 먼저 COCO→가방류 변환 코드를 실행해 /content/items_dataset 를 만들어줘."
assert os.path.isdir(f"{ITEMS_ROOT}/val/images") and os.path.isdir(f"{ITEMS_ROOT}/val/labels"), \
    "items_dataset val 구조가 필요해."
print("✅ items_dataset 확인 OK →", ITEMS_ROOT)


✅ items_dataset 확인 OK → /content/items_dataset


In [8]:
COMBINED_YAML_PATH = f"{BASE}/data_combined.yaml"
yaml_text = f"""\
path: .
train:
  - {HAND_ROOT}/train/images
  - {ITEMS_ROOT}/train/images
val:
  - {HAND_ROOT}/val/images
  - {ITEMS_ROOT}/val/images
nc: 4
names: [hand, backpack, handbag, suitcase]
"""
open(COMBINED_YAML_PATH, "w").write(yaml_text)
print("✅ data.yaml 생성:", COMBINED_YAML_PATH)
print(yaml_text)


✅ data.yaml 생성: /content/data_combined.yaml
path: .
train:
  - /content/hand_dataset/train/images
  - /content/items_dataset/train/images
val:
  - /content/hand_dataset/val/images
  - /content/items_dataset/val/images
nc: 4
names: [hand, backpack, handbag, suitcase]



In [9]:
import glob

def count_labels(lbl_dirs):
    cnt = {0:0,1:0,2:0,3:0}
    for d in lbl_dirs:
        for p in glob.glob(d+"/*.txt"):
            for ln in open(p):
                if ln.strip():
                    cid = int(ln.split()[0])
                    if cid in cnt: cnt[cid]+=1
    return cnt

print("[HAND train]", count_labels([f"{HAND_ROOT}/train/labels"]))
print("[ITEMS train]", count_labels([f"{ITEMS_ROOT}/train/labels"]))
print("[COMBINED train]", count_labels([f"{HAND_ROOT}/train/labels", f"{ITEMS_ROOT}/train/labels"]))


[HAND train] {0: 1499, 1: 0, 2: 0, 3: 0}
[ITEMS train] {0: 0, 1: 5036, 2: 6322, 3: 5041}
[COMBINED train] {0: 1499, 1: 5036, 2: 6322, 3: 5041}


In [10]:
import os, glob

# HAND_ROOT, ITEMS_ROOT 가 앞에서 이미 정의되어 있다는 전제
# hand 오버샘플링을 했다면 train_x1, train_x2 … 자동 포함됨
hand_train_paths = [f"{HAND_ROOT}/train/images"] + \
                   sorted(glob.glob(f"{HAND_ROOT}/train_x*/images"))

train_paths = hand_train_paths + [f"{ITEMS_ROOT}/train/images"]
val_paths   = [f"{HAND_ROOT}/val/images", f"{ITEMS_ROOT}/val/images"]

COMBINED_YAML_PATH = "/content/data_combined.yaml"
yaml_text = "path: .\ntrain:\n" + \
            "".join([f"  - {p}\n" for p in train_paths]) + \
            "val:\n" + \
            "".join([f"  - {p}\n" for p in val_paths]) + \
            "nc: 4\nnames: [hand, backpack, handbag, suitcase]\n"

open(COMBINED_YAML_PATH, "w").write(yaml_text)
print("✅ data_combined.yaml 생성/갱신 완료:\n", yaml_text)


✅ data_combined.yaml 생성/갱신 완료:
 path: .
train:
  - /content/hand_dataset/train/images
  - /content/items_dataset/train/images
val:
  - /content/hand_dataset/val/images
  - /content/items_dataset/val/images
nc: 4
names: [hand, backpack, handbag, suitcase]



In [ ]:
# 1) 충돌 가능 패키지 제거
!pip -q uninstall -y numpy ultralytics opencv-python opencv-contrib-python opencv-python-headless pycocotools fastai thinc spacy >/dev/null

# 2) 호환 조합 "고정" 설치 (Ultralytics 8.3.0 + NumPy 1.26.4 + OpenCV headless 4.11.0.86 + COCO툴즈)
!pip -q install --no-cache-dir numpy==1.26.4 ultralytics==8.3.0 opencv-python-headless==4.11.0.86 pycocotools==2.0.7 >/dev/null

import numpy as _np, os
print("NumPy:", _np.__version__)  # 1.26.4 여야 정상
os.environ["WANDB_DISABLED"] = "true"  # W&B 질문 막기

# 3) 파이썬 커널 강제 재시작(중요!)
os.kill(os.getpid(), 9)


In [1]:
COMBINED_YAML_PATH = "/content/data_combined.yaml"   # 네가 만든 통합 YAML 경로로 맞춰주세요
import os
assert os.path.exists(COMBINED_YAML_PATH), f"YAML이 없어요: {COMBINED_YAML_PATH}"
print("YAML OK:", COMBINED_YAML_PATH)


YAML OK: /content/data_combined.yaml


In [ ]:
import os; os.kill(os.getpid(), 9)

In [1]:
# --- wandb 완전 제거 ---
!pip -q uninstall -y wandb

# --- W&B 비활성화 환경변수(이 셀은 항상 import 전에!) ---
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["WANDB_MODE"] = "offline"
os.environ["WANDB_SILENT"] = "true"


In [2]:
from ultralytics.utils import SETTINGS
SETTINGS.update({'wandb': False})
print("Ultralytics wandb setting:", SETTINGS.get('wandb'))
# 출력이 False면 정상


Ultralytics wandb setting: False


In [4]:
import os, textwrap

# 당신이 만든 폴더 위치로 맞춰주세요
HAND_ROOT  = "/content/hand_dataset"
ITEMS_ROOT = "/content/items_dataset"

COMBINED_YAML_PATH = "/content/data_combined.yaml"

# YAML이 없으면 생성(이미 있다면 건너뜀)
if not os.path.exists(COMBINED_YAML_PATH):
    yaml_text = textwrap.dedent(f"""
    path: /content
    train:
      - {HAND_ROOT}/train/images
      - {ITEMS_ROOT}/train/images
    val:
      - {HAND_ROOT}/val/images
      - {ITEMS_ROOT}/val/images
    nc: 4
    names: [hand, backpack, handbag, suitcase]
    """).strip() + "\n"
    with open(COMBINED_YAML_PATH, "w") as f:
        f.write(yaml_text)

print("YAML @", COMBINED_YAML_PATH)
print(open(COMBINED_YAML_PATH).read())


YAML @ /content/data_combined.yaml
path: .
train:
  - /content/hand_dataset/train/images
  - /content/items_dataset/train/images
val:
  - /content/hand_dataset/val/images
  - /content/items_dataset/val/images
nc: 4
names: [hand, backpack, handbag, suitcase]



In [5]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # 필요시 다른 사전학습 가중치 사용
model.train(
    data=COMBINED_YAML_PATH,
    epochs=20, imgsz=640, batch=8,
    lr0=0.002, weight_decay=0.0005,
    freeze=10,                      # 백본 고정
    mosaic=0.8, mixup=0.05, degrees=5, translate=0.10, scale=0.5, shear=2,
    cache=True, name="hand_items_freeze"
)


New https://pypi.org/project/ultralytics/8.3.178 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.0 🚀 Python-3.11.13 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/data_combined.yaml, epochs=20, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=True, device=None, workers=8, project=None, name=hand_items_freeze4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=10, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_

train: Scanning /content/hand_dataset/train/labels... 9314 images, 1 backgrounds, 0 corrupt: 100%|██████████| 9314/9314 [00:28<00:00, 332.53it/s]


train: New cache created: /content/hand_dataset/train/labels.cache
train: 11.6GB RAM required to cache images with 50% safety margin but only 10.9/12.7GB available, not caching images ⚠️
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.11/dist-packages/ultralytics/data/augment.py:1850: UserWarning: Argument(s) 'quality_lower' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=75, p=0.0),
val: Scanning /content/hand_dataset/val/labels... 422 images, 0 backgrounds, 0 corrupt: 100%|██████████| 422/422 [00:01<00:00, 301.85it/s]


val: New cache created: /content/hand_dataset/val/labels.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


val: Caching images (0.3GB RAM): 100%|██████████| 422/422 [00:07<00:00, 59.08it/s]


Plotting labels to runs/detect/hand_items_freeze4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.002' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 63 weight(decay=0.0), 70 weight(decay=0.0005), 69 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/hand_items_freeze4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/20         0G      1.455       4.16      1.521         23        640:   4%|▍         | 52/1165 [03:55<1:24:00,  4.53s/it]


KeyboardInterrupt: 

In [ ]:
ft = YOLO("runs/detect/hand_items_freeze/weights/best.pt")
ft.train(
    data=COMBINED_YAML_PATH,
    epochs=40, imgsz=640, batch=8,
    lr0=0.001, weight_decay=0.0005,
    freeze=None,                    # 고정 해제
    mosaic=0.6, mixup=0.0, degrees=5, translate=0.10, scale=0.5, shear=2,
    cache=True, name="hand_items_finetune"
)


In [ ]:
ft_val = YOLO("runs/detect/hand_items_finetune/weights/best.pt")
ft_val.val(data=COMBINED_YAML_PATH, imgsz=640)

# Colab이라면:
#from google.colab import files
#files.download("runs/detect/hand_items_finetune/weights/best.pt")
